# 6 Pandas Techniques that Saved My Life

*by [Derek Meegan](https://derekmeegan.com)*

If you work with data in your job or side projects, then you likely spend a vast majority of your time cleaning, manipulating, and transforming your data before you even get to work with it. In fact, it's a popular trope among data scientists that 80% of their time is spent wrangling the data.

Given this reality, you've probably worked with pandas, the essential Python library for data manipulation. I've used pandas extensively for everything from ad-hoc analyses to building production-level data pipelines. Through my experience, I've collected six cardinal techniques that have significantly streamlined my workflow and improved the quality of my code.

Below, I'll explore the techniques and demonstrate how to apply them effectively using the iconic Titanic Dataset.

## Chaining

Chaining is at the center of all the techniques described below. It's a method that allows you to apply multiple operations to your data in a single, continuous statement, creating a streamlined pipeline that looks more like a cohesive recipe than a series of separate steps. This style of pandas was popularized by Matt Harrison, a Python and data science trainer and author of [Effective Pandas](https://www.amazon.com/Effective-Pandas-Patterns-Manipulation-Treading/dp/B09MYXXSFM) (highly recommend).

Chaining does not resemble traditional programming, which aims to make operations independent and functions small and simple. Instead, chaining represents ETL workflows more realistically, where data is gradually cleansed or augmented through a series of operations. This approach can be intimidating for beginners due to the potential length and complexity of workflows. However, chaining quickly becomes an easy and intuitive approach to creating robust data pipelines by understanding a few basics and breaking down workflows step by step.

Below I have created a sample workflow for performing a common set of operations on the dataset. I used variable reassignment in the first snippet and chaining in the second. Which one looks cleaner to you?

In [33]:
import pandas as pd

In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

df['Age'] = df.Age.ffill()
df['Sex'] = df.Sex.map({'male': 1, 'female': 0})

survived_by_class = df.groupby('Pclass').sum()['Survived']

survived_by_class = survived_by_class.sort_values()
survived_by_class

Pclass
2     87
3    119
1    136
Name: Survived, dtype: int64

In [35]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .assign(
        Age = lambda df_: df_.Age.ffill(),
        Sex = lambda df_: df_.Sex.map({'male': 1, 'female': 0})
    )
    .groupby('Pclass').sum()['Survived']
    .sort_values()
)

Pclass
2     87
3    119
1    136
Name: Survived, dtype: int64

Notice how we did not have to create variables in the chain version! If you were executing this pipeline in a Jupyter Notebook, you could directly run the cell to inspect the results. In production, you would need to save the results to a variable, but you eliminate four other variable instantiations, resulting in cleaner and more maintainable code.

To maintain visual clarity when chaining, arrange your pipeline code hierarchically, where each additional indent corresponds to a deeper level in the pipeline. This becomes increasingly crucial as the workflow becomes more complex. Additionally, note the use of "lambda" functions in the .assign method. While they may seem daunting at first, these lambdas simply reference the preceding dataframe they are attached to, which is particularly useful when applying transformations to grouped or filtered data or when creating multiple interdependent columns.

Now that we understand the importance and advantages of chaining, let's explore some techniques that leverage this approach to tackle complex tasks easily.

## Inspecting Duplicated Rows

Pandas' built-in drop_duplicates function is useful for removing duplicate rows, but it doesn't show us the duplicated rows themselves. To identify duplicates, we use the .duplicated method, which returns a boolean series where rows are True if they are duplicates of a preceding row. However, this often leads to the creation of clunky code:

In [36]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

duplicated = df.duplicated()
duplicated_df = df[duplicated]

While this achieves our goal, we can improve our approach. First, we can replace vanilla boolean indexing with .loc , allowing us to create dynamic filtering expressions.

In [37]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .loc[lambda row: row.duplicated()]
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


And voila! We filtered for the duplicated rows without variable reassignment. While you can use .drop_duplicates like we mentioned above to drop the duplicated rows, you can also filter them out using the same expression above with a ~ operator prepended to the row.duplicated filter.

In [38]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .loc[lambda row: ~row.duplicated()]
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


The ~ operator reverses the boolean conditions of the filter statement, such that only rows that were not duplicated would be returned. This is handy when you want to inspect your data and quickly peek at the duplicated and non-duplicated rows.

## Value selection using .loc

One aspect of pandas I have found tricky is accessing a specific value in an individual cell. While we often use pandas to operate on rows or columns, sometimes we need to extract individual values. The general technique for doing this typically looks like the following:

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

passenger = df.loc[lambda row: row.Name == 'Montvila, Rev. Juozas']
ages = passenger.Age
passenger_age = ages.iloc[0]
passenger_age

27.0

While this does work, the .iloc property can feel a bit hacky. Fortunately, with a bit of preparation before using .loc method, we can access values directly. The key is to set the column you are filtering on as the index of the dataframe. This way, you can use the name as the first accessor in the .loc statement and then specify the desired column as the second accessor. 

Note that if the column you set as the index contains duplicate values, you may receive multiple rows as a result of your .loc statement. Therefore, ensure that duplicates are either handled or not present in the column you set as the index.

In [40]:
passenger_age = (
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .set_index('Name')
    .loc['Montvila, Rev. Juozas', 'Age']
)
passenger_age

27.0

The following techniques all revolve around .pipe, one of the handiest methods in pandas. It allows you to chain any custom function, whether predefined or expressed through lambda, onto a dataframe or series. However, be cautious: the result of a dataframe passed through a pipe function is not always a dataframe itself. Depending on how the function returns, you may not be able to chain additional pandas transformations. Let's take a look at how it works.

## Pipe Ternary

Pandas is excellent at performing row, column, and table-oriented transformations on data, but ironically has poor support for conditional operations. One workaround I have leveraged is using ternary functions within lambdas in the .pipe method. Let's say you are creating a pipeline for the Titanic dataset and it expects a column called "Cabin". However, you cannot guarantee that the column will always be present. You may do something like this:

In [41]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

columns = df.columns
if 'Cabin' not in columns:
    df['Cabin'] = None

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


This works; however, it breaks your chain! For a one-off operation, this might be acceptable, but if multiple such instances occur in a larger pipeline, it quickly becomes unreadable. Instead, we can define a simple lambda function within the pipe to perform the ternary operation:

In [42]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .pipe(
        lambda df_: 
            df_ if 'Cabin' in df_.columns
            else df_.assign(Cabin = None)

    )
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Using .pipe, we create a custom ternary function that checks if "Cabin" already exists in the dataframe's columns. If it does, the function directly returns the dataframe; if not, it assigns the column. Now, let's take a look at something a bit fancier…

## Apply Transformation to a Subset of Columns

This is by far the most useful technique in my day-to-day work, as I often deal with datasets comprising multiple data types, with similar columns requiring the same transformations. Take the Titanic dataset, for example. There are null values across various columns, but let's say we only want to forward-fill the "Age" and "Sex" columns. How would we do that? One way is by using the .assign method and lambda functions to overwrite the original columns:

In [43]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .assign(
        Age = lambda df_: df_.Age.ffill(),
        Sex = lambda df_: df_.Sex.ffill()
    )
    
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,19.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


But what if, instead of transforming two columns, we want to transform ten? Typing the same transformation repeatedly becomes tedious, and if you need to change the transformation, you'll have to update it ten times. Luckily, there's a better way using .pipe, lambda, and a little Python unpacking magic:

In [44]:
(
    pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
    .pipe(
        lambda df_: (
            df_.assign(
                **df_[['Sex', 'Age']].ffill()
            )
        )
    )
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,19.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


This may look confusing at first, but like any chained flow, let's break it down step by step. First, the lambda function in the .pipe method passes the dataframe into a new flow (yes, a flow within a flow). In this flow, we unpack a smaller dataframe (comprising only the columns we want to transform) from the original dataframe. Using this technique, we can apply .ffill to the smaller dataframe, targeting only the desired columns, and then unpack these columns directly back onto the original dataframe. This approach is not only syntactically clearer but also provides performance optimization, as .ffill is called only once! 

Next, we will explore how .pipe can help us flow our prepared data directly into visualization libraries.

## Piping Transformed Data Directly into Plotly

While I prefer Plotly for its intuitiveness and visual appeal, this strategy also works with Matplotlib, Seaborn, and other graphing libraries that interface directly with pandas dataframes and NumPy series. Typically, we perform our data operations first and then create a secondary function for visualizing the results. However, when working with data, the speed of iteration is important. Ideally, we could append visualizations directly to our chained flow to better inspect the results of our analysis. Using the .pipe method, we can do just that. For example, let's say I want to take a glance at the distribution of fare prices aboard the Titanic:

In [45]:
import plotly.express as px

(
    df
    .dropna(subset = 'Fare')
    .pipe(
        px.histogram,
        x = 'Fare',
        nbins = 15
    )
    .update_layout(
        title = 'Titanic Dataset - Distribution of Fares',
        template = 'plotly_dark',
        yaxis = dict(title = 'Count'),
        height = 750,
        width = 1250,
    )
)

In this flow, I do not use a lambda function but instead pass the desired function directly as the first argument in the .pipe method. The subsequent arguments are associated with the px.histogram function and pandas smartly passes these as additional keyword arguments. The result of the .pipe method is a Plotly chart, allowing me to directly append Plotly methods to the chain, seamlessly merging my pipeline and visualization.

And there you have it - six life-saving techniques for writing better and more efficient pandas code. In each example, I started the flow with the original dataset. This is intentional, as it's generally better to read the data directly from the source for each analysis, especially in notebook environments. This approach prevents previous data transformations from altering the results of your pipeline and ensures your pipelines remain reproducible, even if the notebook outputs are refreshed or removed.

For more about me, my projects, and other work, visit my [website](https://derekmeegan.com).